In [1]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

### 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成
### 【問題2】1次元畳み込み後の出力サイズの計算
### 【問題3】小さな配列での1次元畳み込み層の実験

In [2]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])


In [3]:
# print(sum(x[0:3] * w)+b)
# print(sum(x[1:4] * w)+b)

a = []
for s in range(len(w) - 1):
    a.append((x[s:w.shape[0]+s] @ w) + b)

print(a)


[array([35]), array([50])]


In [4]:
n_out = 0 # 求める値。出力のサイズ（特徴量の数）
n_in = x.shape[0] # 入力のサイズ（特徴量の数）
p = 1 # ある方向へのパディング
f = 1 # フィルタのサイズ
s = 1 # ストライドのサイズ

n_out = ((n_in + 2 * p - f) / s ) + 1
print(n_out)

6.0


In [5]:
delta_a = np.array([10, 20])

delta_b = np.sum(delta_a)
a = []
# c = []
for i in range(len(w)):
    a.append(delta_a @ x[i:len(delta_a)+i])
#    c.append(sum(delta_a * x[i:len(delta_a)+i])) # 計算結果として上と同じになる。意味的にはこちら
delta_w = np.array(a)
# delta_w1 = np.array(c)
# print(c)

# ここ(バックプロパゲーション)の処理を確認
# b = []
# new_w = np.insert(w[::-1], 0, 0)
# new_w = np.append(new_w, 0)
# for s in range(len(new_w) -1):
#     b.append(new_w[s:len(delta_a)+s] @ delta_a)
# delta_x = np.array(b[::-1])

b = []
print(len(delta_w))
print(delta_w)
for da_cnt_1 in range(len(delta_a)):
    for w_cnt_1 in range(len(delta_w)):
        if (da_cnt_1 == 0) & (w_cnt_1 == 0):
            b.append(delta_a[da_cnt_1] * w[w_cnt_1])
        elif da_cnt_1 == 1:
            b.append(delta_a[da_cnt_1] * w[len(delta_w)-1])
            break
        else:
            b.append(
                (delta_a[da_cnt_1] * w[w_cnt_1]) 
                + 
                (delta_a[da_cnt_1 + 1] * w[w_cnt_1 - 1]))
delta_x = np.array(b)

print("delta_b:{}".format(delta_b))
print("delta_w:{}".format(delta_w))
print("delta_x:{}".format(delta_x))
print("delta_x.shape:{}".format(delta_x.shape))

print("delta_x[0]:{}".format(delta_a[0] * w[0]))
print("delta_x[1]:{}".format((delta_a[0] * w[1]) + (delta_a[1] * w[0])))
print("delta_x[2]:{}".format((delta_a[0] * w[2]) + (delta_a[1] * w[1])))
print("delta_x[3]:{}".format(delta_a[1] * w[2]))


3
[ 50  80 110]
delta_b:30
delta_w:[ 50  80 110]
delta_x:[ 30 110 170 140]
delta_x.shape:(4,)
delta_x[0]:30
delta_x[1]:110
delta_x[2]:170
delta_x[3]:140


In [6]:
def output_size_calculation(n_in, F, P=0, S=1):
    n_out = int((n_in + 2*P - F) / S + 1)
    return n_out


In [7]:
class SimpleConv1d():

    def forward(self, x, w, b):
        a = []
        for i in range(len(w) - 1):
            a.append((x[i:i+len(w)] @ w) + b[0])
        return np.array(a)
    
    def backward(self, x, w, da):
        db = np.sum(da)
        dw = []
        for i in range(len(w)):
            dw.append(da @ x[i:i+len(da)])
        dw = np.array(dw)
        dx = []
        new_w = np.insert(w[::-1], 0, 0)
        new_w = np.append(new_w, 0)
        for i in range(len(new_w)-1):
            dx.append(new_w[i:i+len(da)] @ da)
        dx = np.array(dx[::-1])
        return db, dw, dx


### 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成

In [8]:
class GetMiniBatch:

    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(int)
        
    def __len__(self):
        return self._stop
    
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1] 
    
    def __iter__(self):
        self._counter = 0
        return self
    
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]


In [9]:
class SimpleInitializer:

    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, *shape):
        W = self.sigma * np.random.randn(*shape)
        return W
    
    def B(self, *shape):
        B = self.sigma * np.random.randn(*shape)
        return B


In [10]:
class SGD:
    
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, layer):
        layer.W -= self.lr * layer.dW / len(layer.Z)
        layer.B -= self.lr * layer.dB / len(layer.Z)
        return layer


In [11]:
class AdaGrad:

    def __init__(self, lr):
        self.lr = lr 
    
    def update(self, layer):
        layer.HW += layer.dW * layer.dW
        layer.HB += layer.dB * layer.dB
        delta = 1e-7 
        layer.W -= self.lr * layer.dW / (np.sqrt(layer.HW) + delta) / len(layer.Z)
        layer.B -= self.lr * layer.dB / (np.sqrt(layer.HB) + delta) / len(layer.Z)
        return layer


In [12]:
class softmax:

    def forward(self, A): 
        Z = np.exp(A) / np.sum(np.exp(A), axis=1).reshape(-1, 1)
        return Z
        
    def backward(self, Z, y):
        dA = Z - y
        loss = - np.sum(y * np.log(Z)) / len(y)
        return dA, loss


In [13]:
class Conv1d:
    
    def __init__(self,
                 b_size,
                 initializer,
                 optimizer,
                 n_in_channels=1,
                 n_out_channels=1,
                 pa=0):
        self.b_size = b_size
        self.optimizer = optimizer
        self.pa = pa
        self.W = initializer.W(n_out_channels, n_in_channels, b_size)
        # self.W = initializer.W(n_out_channels, n_in_channels)
        self.B = initializer.B(n_out_channels)
        self.n_in_channels = n_in_channels
        self.n_out_channels = n_out_channels
        self.n_out = None
        
    def forward(self, X):
        #Forward Propagation
        # print("==================================================")
        # print("X:{}".format(X))
        self.n_in = X.shape[-1]
        self.n_out = output_size_calculation(self.n_in, self.b_size, self.pa)
        X = X.reshape(self.n_in_channels, self.n_in)
        self.X = np.pad(X, ((0,0), ((self.b_size-1), 0)))
        self.X1 = np.zeros((self.n_in_channels, self.b_size, self.n_in+(self.b_size-1)))
        # print("self.n_in:{}".format(self.n_in))
        # print("self.n_out:{}".format(self.n_out))
        # print("X:{}".format(X))
        # print("self.X:{}".format(self.X))
        for i in range(self.b_size):
            self.X1[:, i] = np.roll(self.X, -i, axis=-1)
            # print("self.X1[:, i]:{}".format(self.X1[:, i]))
        # print("==================================================")
        # print("self.X1:{}".format(self.X1))
        # print("self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa].shape:{}".format((self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa]).shape))
        # print("self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa]:{}".format(self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa]))
        # print("self.W[:, :, :, np.newaxis].shape:{}".format((self.W[:, :, :, np.newaxis]).shape))
        # print("self.W[:, :, :, np.newaxis]:{}".format(self.W[:, :, :, np.newaxis]))
        A = np.sum(self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa]*self.W[:, :, :, np.newaxis], axis=(1, 2)) + self.B.reshape(-1,1)
        # print("==================================================")
        # print("A:{}".format(A))
        return A
    
    def backward(self, dA):
        #Back Propagation
        self.dW = np.sum(np.dot(dA, self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa, np.newaxis]), axis=-1)
        self.dB = np.sum(dA, axis=1)
        self.dA = np.pad(dA, ((0,0), (0, (self.b_size-1))))
        self.dA1 = np.zeros((self.n_out_channels, self.b_size, self.dA.shape[-1]))
        for i in range(self.b_size):
            self.dA1[:, i] = np.roll(self.dA, i, axis=-1)
        dX = np.sum(self.W@self.dA1, axis=0)
        self.optimizer.update(self)
        return dX

    def output_size_calculation(n_in, F, P=0, S=1):
        n_out = int((n_in + 2*P - F) / S + 1)
        return n_out

In [14]:
"""
b_size : フィルターサイズ
initializer : 初期化クラス
optimizer : 最適化手法クラス
n_in_channels : 入力チャンネル数
n_out_channels : 出力チャンネル数
pa : パディング数
"""
test = Conv1d(b_size=3,
              initializer=SimpleInitializer(0.01),
              optimizer=SGD(0.01),
              n_in_channels=2,
              n_out_channels=3,
              pa=0)


In [15]:
# x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
# w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
# b = np.array([1, 2, 3]) # （出力チャンネル数）

x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) 
test.W = np.ones((3, 2, 3), dtype=float)
test.B = np.array([1, 2, 3], dtype=float)


In [16]:
a = test.forward(x)
print(a)


[[16. 22.]
 [17. 23.]
 [18. 24.]]


【問題8】学習と推定

In [50]:
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import math


class Sigmoid:
    
    def forward(self, A):
        self.A = A
        return self.sigmoid(A)
    
    def backward(self, dZ):
        _sig = self.sigmoid(self.A)
        return dZ * (1 - _sig)*_sig
    
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-X))

class Tanh:
    
    def forward(self, A):
        self.A = A
        return np.tanh(A)
    
    def backward(self, dZ):
        return dZ * (1 - (np.tanh(self.A))**2)

class Softmax:
    
    def forward(self, X):
        self.Z = np.exp(X) / np.sum(np.exp(X), axis=1).reshape(-1,1)
        return self.Z
    
    def backward(self, Y):
        self.loss = self.loss_func(Y)
        return self.Z - Y
    
    def loss_func(self, Y, Z=None):
        if Z is None:
            Z = self.Z
        return (-1)*np.average(np.sum(Y*np.log(Z), axis=1))

class ReLU:
    
    def forward(self, A):
        self.A = A
        return np.clip(A, 0, None)
    
    def backward(self, dZ):
        return dZ * np.clip(np.sign(self.A), 0, None)

class FC:

    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)
        
    def forward(self, X):
        self.X = X
        A = X@self.W + self.B
        return A
    
    def backward(self, dA):
        dZ = dA@self.W.T
        self.dB = np.sum(dA, axis=0)
        self.dW = self.X.T@dA
        self.optimizer.update(self)
        return dZ

class XavierInitializer:
    
    def W(self, n_nodes1, n_nodes2):
        self.sigma = math.sqrt(1 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B
    
class HeInitializer():
    
    def W(self, n_nodes1, n_nodes2):
        self.sigma = math.sqrt(2 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B

class SGD:

    def __init__(self, lr):
        self.lr = lr
    
    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB
        return

class AdaGrad:
    
    def __init__(self, lr):
        self.lr = lr
        self.HW = 1
        self.HB = 1
    
    def update(self, layer):
        self.HW += layer.dW**2
        self.HB += layer.dB**2
        layer.W -= self.lr * np.sqrt(1/self.HW) * layer.dW
        layer.B -= self.lr * np.sqrt(1/self.HB) * layer.dB

class Conv1d_Arbitrary_Strides:
    
    def __init__(self, b_size, initializer, optimizer, n_in_channels=1, n_out_channels=1, pa=0, stride=1):
        self.b_size = b_size
        self.optimizer = optimizer
        self.pa = pa
        self.stride = stride
        self.W = initializer.W(n_out_channels, n_in_channels, b_size)
        self.B = initializer.B(n_out_channels)
        self.n_in_channels = n_in_channels
        self.n_out_channels = n_out_channels
        self.n_out = None
        
    def forward(self, X):
        
        #Forward propagation while takin into account the strides
        self.n_samples = X.shape[0]
        self.n_in = X.shape[-1]
        self.n_out = output_size_calculation(self.n_in, self.b_size, self.pa, self.stride)
        X = X.reshape(self.n_samples, self.n_in_channels, self.n_in)
        self.X = np.pad(X, ((0,0), (0,0), ((self.b_size-1), 0)))
        self.X1 = np.zeros((self.n_samples, self.n_in_channels, self.b_size, self.n_in+(self.b_size-1)))
        for i in range(self.b_size):
            self.X1[:, :, i] = np.roll(self.X, -i, axis=-1)
        A = np.sum(self.X1[:, np.newaxis, :, :, self.b_size-1-self.pa:self.n_in+self.pa:self.stride]*self.W[:, :, :, np.newaxis], axis=(2, 3)) + self.B.reshape(-1,1)
        return A
    
    def backward(self, dA):
        
        #Back propagation while takin into account the strides
        self.dW = np.sum(dA[:, :, np.newaxis, np.newaxis]*self.X1[:, np.newaxis, :, :, self.b_size-1-self.pa:self.n_in+self.pa:self.stride], axis=(0, -1))
        self.dB = np.sum(dA, axis=(0, -1))
        self.dA = np.pad(dA, ((0,0), (0,0), (0, (self.b_size-1))))
        self.dA1 = np.zeros((self.n_samples, self.n_out_channels, self.b_size, self.dA.shape[-1]))
        for i in range(self.b_size):
            self.dA1[:, :, i] = np.roll(self.dA, i, axis=-1)
        dX = np.sum(self.W[:, :, :, np.newaxis]*self.dA1[:, :, np.newaxis], axis=(1,3))
        self.optimizer.update(self)
        return dX

def output_size_calculation(n_in, F, P=0, S=1):
    n_out = int((n_in + 2*P - F) / S + 1)
    return n_out



In [51]:
class ScratchCNNClassifier:
    
    def __init__(self, num_epoch=10, lr=0.01, batch_size=20, n_features=784, n_nodes1=400, n_nodes2=200, n_output=10, verbose=True, Activater=Tanh, Optimizer=AdaGrad):
        self.num_epoch = num_epoch
        self.lr = lr
        self.verbose = verbose  
        self.batch_size = batch_size 
        self.n_features = n_features 
        self.n_nodes2 = n_nodes2 
        self.n_output = n_output 
        self.Activater = Activater
        if Activater == Sigmoid or Activater == Tanh:
            self.Initializer = XavierInitializer
        elif Activater == ReLU:
            self.Initializer = HeInitializer
        self.Optimizer = Optimizer
    
    def fit(self, X, y, X_val=None, y_val=None):
        self.val_enable = False
        if X_val is not None:
            self.val_enable = True
        
        #Preparing the Convolutional layer and the Fullty Connected layers
        self.Conv1d_Arbitrary_Strides = Conv1d_Arbitrary_Strides(b_size=7, initializer=SimpleInitializer(0.01), optimizer=self.Optimizer(self.lr), n_in_channels=1, n_out_channels=1, pa=3, stride=2)
        self.Conv1d_Arbitrary_Strides.n_out = output_size_calculation(X.shape[-1], self.Conv1d_Arbitrary_Strides.b_size, self.Conv1d_Arbitrary_Strides.pa, self.Conv1d_Arbitrary_Strides.stride)
        self.activation1 = self.Activater()
        self.FC2 = FC(1*self.Conv1d_Arbitrary_Strides.n_out, self.n_nodes2, self.Initializer(), self.Optimizer(self.lr))
        self.activation2 = self.Activater()
        self.FC3 = FC(self.n_nodes2, self.n_output, self.Initializer(), self.Optimizer(self.lr))
        self.activation3 = Softmax()
        
        self.loss = []
        self.loss_epoch = [self.activation3.loss_func(y, self.forward_propagation(X))]
        
        #Going through the number of epochs, doing forward and back propagations and recording the loss.
        for _ in range(self.num_epoch):
            #Instantiation of the GetMiniBatch class
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch_size)
            self.iter = len(get_mini_batch)
            for mini_X, mini_y in get_mini_batch:
                self.forward_propagation(mini_X)
                self.back_propagation(mini_X, mini_y)
                self.loss.append(self.activation3.loss)
            self.loss_epoch.append(self.activation3.loss_func(y, self.forward_propagation(X)))
    
    #Doing predictions. Using np.argmax() in order to get the indice of the highest value 
    def predict(self, X):
        return np.argmax(self.forward_propagation(X), axis=1)
    
    def forward_propagation(self, X):
        
        #Forward Propagation from the COnvolutional layer to the Fully Connected layers
        A1 = self.Conv1d_Arbitrary_Strides.forward(X)
        A1 = A1.reshape(A1.shape[0], A1.shape[-1])
        Z1 = self.activation1.forward(A1)
        A2 = self.FC2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.FC3.forward(Z2)
        Z3 = self.activation3.forward(A3)
        return Z3
        
    def back_propagation(self, X, y_true):
        
        #back Propagation from the Fully Connected layers to the Convolutional layer
        dA3 = self.activation3.backward(y_true) 
        dZ2 = self.FC3.backward(dA3)
        dA2 = self.activation2.backward(dZ2)
        dZ1 = self.FC2.backward(dA2)
        dA1 = self.activation1.backward(dZ1)
        dA1 = dA1[:, np.newaxis]
        dZ0 = self.Conv1d_Arbitrary_Strides.backward(dA1) 


In [52]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)


X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

test3 = ScratchCNNClassifier(num_epoch=20, lr=0.01, batch_size=20, n_features=784, n_nodes1=400, n_nodes2=400, n_output=10, verbose=True, Activater=Tanh, Optimizer=SGD)
test3.fit(X_train_, y_train_)


<ipython-input-52-8d0a3d370e29>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train = X_train.astype(np.float)
<ipython-input-52-8d0a3d370e29>:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test = X_test.astype(np.float)


In [53]:
y_pred = test3.predict(X_test)
accuracy_score(y_test, y_pred)

0.9803